In [1]:
import multiprocessing
import time

def worker(interval):
    n = 5
    while n > 0:
        print("The time is {0}".format(time.ctime()))
        time.sleep(interval)
        n -= 1
        
if __name__ == "__main__":
    p = multiprocessing.Process(target = worker, args = (3,))
    p.start()
    print("p.pid:", p.pid)
    print("p.name:", p.name)
    print("p.is_alive", p.is_alive())

p.pid: 768
p.name: Process-1
p.is_alive True
The time is Tue Apr  2 23:30:53 2019
The time is Tue Apr  2 23:30:56 2019
The time is Tue Apr  2 23:30:59 2019
The time is Tue Apr  2 23:31:02 2019
The time is Tue Apr  2 23:31:05 2019


In [2]:
import multiprocessing
import time

def worker_1(interval):
    print("worker_1")
    time.sleep(interval)
    print("end worker_1")
    
def worker_2(interval):
    print("worker_2")
    time.sleep(interval)
    print("end worker_2")
        
def worker_3(interval):
    print("worker_3")
    time.sleep(interval)
    print("end worker_3")
    
if __name__ == "__main__":
    p1 = multiprocessing.Process(target = worker_1, args = (2,))
    p2 = multiprocessing.Process(target = worker_2, args = (3,))
    p3 = multiprocessing.Process(target = worker_3, args = (4,))
    
    p1.start()
    p2.start()
    p3.start()
    
    print("The number of CPU is:" + str(multiprocessing.cpu_count()))
    for p in multiprocessing.active_children():
        print("child p.name" + p.name + "\tp.id" + str(p.pid))
    print("END !!!!!!!!!!!!!!")

worker_1
The number of CPU is:4
child p.nameProcess-4	p.id771
child p.nameProcess-1	p.id768
child p.nameProcess-3	p.id770
child p.nameProcess-2	p.id769
END !!!!!!!!!!!!!!
worker_2
worker_3
end worker_1
end worker_2
end worker_3


In [3]:
import multiprocessing
import time

class ClockProcess(multiprocessing.Process):
    def __init__(self, interval):
        multiprocessing.Process.__init__(self)
        self.interval = interval
    def run(self):
        n = 5
        while n > 0:
            print("the time is {0}".format(time.ctime()))
            time.sleep(self.interval)
            n -= 1
    
if __name__ == "__main__":
    p = ClockProcess(3)
    p.start()

the time is Tue Apr  2 23:30:53 2019
the time is Tue Apr  2 23:30:56 2019
the time is Tue Apr  2 23:30:59 2019
the time is Tue Apr  2 23:31:02 2019
the time is Tue Apr  2 23:31:05 2019


In [4]:
import multiprocessing
import time

def worker(interval):
    print("worker start:{0}".format(time.ctime()))
    time.sleep(interval)
    print("worker end:{0}".format(time.ctime()))
    
if __name__ == "__main__":
    p = multiprocessing.Process(target = worker, args = (3,))
    p.daemon = True
    p.start()
    p.join()
    print("end!")

worker start:Tue Apr  2 23:30:53 2019
worker end:Tue Apr  2 23:30:56 2019
end!


In [5]:
import multiprocessing
import sys

def worker_with(lock, f):
    with lock:
        fs = open(f, "a+")
        n = 10
        while n > 1:
            fs.write("Locked acquired via with\n")
            n -= 1
        fs.close()

def worker_no_with(lock, f):
    lock.acquire()
    try:
        fs = open(f, "a+")
        n = 10
        while n > 1:
            fs.write("Lock acquired via directly\n")
            n -= 1
        fs.close()
    finally:
        lock.release()
        
if __name__ == "__main__":
    lock = multiprocessing.Lock()
    f = "file.txt"
    w = multiprocessing.Process(target = worker_with, args=(lock, f))
    nw = multiprocessing.Process(target = worker_no_with, args=(lock, f))
    w.start()
    nw.start()
    print("end")

end


In [6]:
import multiprocessing
import time

def worker(s, i):
    s.acquire()
    print(multiprocessing.current_process().name + "acquire")
    time.sleep(i)
    print(multiprocessing.current_process().name + "release\n")
    s.release()
    
if __name__ == "__main__":
    s = multiprocessing.Semaphore(2)
    for i in range(5):
        p = multiprocessing.Process(target = worker, args = (s, i*2))
        p.start()

Process-9acquire
Process-9release
Process-10acquire

Process-11acquire
Process-10release

Process-12acquire
Process-11release

Process-13acquire
Process-12release

Process-13release



In [7]:
import multiprocessing
import time

def wait_for_event(e):
    print("wait_for_event: starting")
    e.wait()
    print("wait_for_event: e.is_set()->" + str(e.is_set()))
    
def wait_for_event_timeout(e, t):
    print("worker_for_event_timeout: starting")
    e.wait(t)
    print("worker_for_event_timeout: e.is_set()->" + str(e.is_set()))
    
if __name__ == "__main__":
    e = multiprocessing.Event()
    w1 = multiprocessing.Process(name = "block",
                                target = wait_for_event,
                                args = (e, ))
    
    w2 = multiprocessing.Process(name = "no-block",
                                target = wait_for_event_timeout,
                                args = (e, 2))
    w1.start()
    w2.start()
    time.sleep(3)
    
    e.set()
    print("main:event is set")

wait_for_event: starting
worker_for_event_timeout: starting
worker_for_event_timeout: e.is_set()->False
main:event is set
wait_for_event: e.is_set()->True


In [8]:
import multiprocessing

def writer_proc(q):
    try:
        q.put(1, block = False)
    except:
        pass

def reader_proc(q):
    try:
        print(q.get(block = False))
    except:
        pass
    
if __name__ == "__main__":
    q = multiprocessing.Queue()
    writer = multiprocessing.Process(target = writer_proc, args = (q, ))
    writer.start()
    
    reader = multiprocessing.Process(target = reader_proc, args=(q, ))
    reader.start()
    
    reader.join()
    writer.join()

1


In [9]:
import multiprocessing
import time

def proc1(pipe):
    while True:
        for i in range(10000):
            print("send: %s" %(i))
            pipe.send(i)
            time.sleep(1)

def proc2(pipe):
    while True:
        print("proc2 rev:", pipe.recv())
        time.sleep()

def proc3(pipe):
    while True:
        print("proc3 rev", pipe.recv())
        time.sleep(1)

if __name__ == "__main__":
    pipe = multiprocessing.Pipe()
    p1 = multiprocessing.Process(target = proc1, args = (pipe[0], ))
    p2 = multiprocessing.Process(target = proc2, args = (pipe[1], ))

    p1.start()
    p2.start()
    
    p1.join()
    p2.join()

send: 0
proc2 rev: 0


Process Process-19:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-9-cf16f6ce0df6>", line 14, in proc2
    time.sleep()
TypeError: sleep() takes exactly one argument (0 given)


send: 1
send: 2
send: 3
send: 4
send: 5
send: 6
send: 7
send: 8
send: 9
send: 10
send: 11
send: 12
send: 13
send: 14
send: 15
send: 16
send: 17
send: 18
send: 19
send: 20
send: 21
send: 22
send: 23
send: 24
send: 25


Process Process-18:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-9-cf16f6ce0df6>", line 9, in proc1
    time.sleep(1)
KeyboardInterrupt


KeyboardInterrupt: 

In [10]:
import multiprocessing
import time

def func(msg):
    print("msg:", msg)
    time.sleep(3)
    print("end")

if __name__=='__main__':
    pool = multiprocessing.Pool(processes = 3)
    print("--------")
    for i in range(4):
        msg = "hello %d" %(i)
        pool.apply_async(func, (msg, ))
    print("Mark~ Mark~~~~~~~~~~~~~~")
    pool.close()
    pool.join()
    print("Sub-process(es) done.")

msg: hello 1
msg: hello 0
msg: hello 2
--------
Mark~ Mark~~~~~~~~~~~~~~
end
end
end
msg: hello 3
end
Sub-process(es) done.
